In [1]:
"""
SNR-to-SNR DA using ResNet
"""

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import funcs
import jan
import coral
import star
import mcd
import dann
import base
import plots
import os
import seaborn as sns
import matplotlib.pyplot as plt
from torch.autograd import Function

#%% ResNet block
class ResidualBlock1D(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        """
        A basic residual block for 1D convolutions.
        """
        super(ResidualBlock1D, self).__init__()
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm1d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm1d(out_channels)
        self.downsample = downsample

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        
        if self.downsample is not None:
            identity = self.downsample(x)
        
        out += identity
        out = self.relu(out)
        return out

#%% Base
class DeepResNet(nn.Module):
    """
    A deep ResNet classifier for 1D signals.
    
    This network consists of:
      - An initial convolution + BN + ReLU + maxpool block.
      - Four residual layers (with increasing feature channels).
      - Global average pooling to obtain a fixed–length feature vector.
      - A bottleneck fully connected layer mapping to 512–dimensions.
      - A small classifier head (MLP) mapping to the desired output classes.
    """
    def __init__(self, output_dim=7):
        super(DeepResNet, self).__init__()
        # Initial convolutional block
        self.conv1 = nn.Conv1d(in_channels=2, out_channels=64, kernel_size=7,
                               stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm1d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool1d(kernel_size=3, stride=2, padding=1)
        
        # Residual layers
        self.layer1 = self._make_layer(in_channels=64, out_channels=64, blocks=2, stride=1)
        self.layer2 = self._make_layer(in_channels=64, out_channels=128, blocks=2, stride=2)
        self.layer3 = self._make_layer(in_channels=128, out_channels=256, blocks=2, stride=2)
        self.layer4 = self._make_layer(in_channels=256, out_channels=512, blocks=2, stride=2)
        
        # Global average pooling to obtain a fixed-length feature vector
        self.avgpool = nn.AdaptiveAvgPool1d(1)
        # Bottleneck fully-connected layer: maps 512-dim to 512-dim features
        self.fc_bottleneck = nn.Linear(512, 512)
        
        # Classifier head (MLP)
        self.fc1 = nn.Linear(512, 256)
        self.bn_fc = nn.BatchNorm1d(256)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(256, output_dim)
        
    def _make_layer(self, in_channels, out_channels, blocks, stride):
        """
        Creates a sequential layer composed of multiple residual blocks.
        If the stride is not 1 or the number of channels change,
        a downsampling layer is used.
        """
        downsample = None
        if stride != 1 or in_channels != out_channels:
            downsample = nn.Sequential(
                nn.Conv1d(in_channels, out_channels, kernel_size=1,
                          stride=stride, bias=False),
                nn.BatchNorm1d(out_channels)
            )
        layers = []
        layers.append(ResidualBlock1D(in_channels, out_channels, stride, downsample))
        for _ in range(1, blocks):
            layers.append(ResidualBlock1D(out_channels, out_channels))
        return nn.Sequential(*layers)
    
    def forward(self, x):
        # x: (batch_size, 2, length)
        x = self.conv1(x)    # (B, 64, L/2)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)  # (B, 64, L/4)
        
        x = self.layer1(x)   # (B, 64, ?)
        x = self.layer2(x)   # (B, 128, ?)
        x = self.layer3(x)   # (B, 256, ?)
        x = self.layer4(x)   # (B, 512, ?)
        
        x = self.avgpool(x)  # (B, 512, 1)
        x = x.squeeze(-1)    # (B, 512)
        x = self.fc_bottleneck(x)  # (B, 512)
        
        # Classification head
        x = self.fc1(x)
        x = self.bn_fc(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

#%% DANN
class GradReverse(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x, lambda_):
        ctx.lambda_ = lambda_
        return x.view_as(x)
    
    @staticmethod
    def backward(ctx, grad_output):
        return grad_output.neg() * ctx.lambda_, None

def grad_reverse(x, lambda_=1.0):
    return GradReverse.apply(x, lambda_)

class DANN_F(nn.Module):
    """
    Deep ResNet feature extractor for DANN.
    
    This network accepts a 2–channel 1D signal and outputs a 512–dimensional feature vector.
    """
    def __init__(self):
        super(DANN_F, self).__init__()
        # Initial convolutional block
        self.conv1 = nn.Conv1d(in_channels=2, out_channels=64, kernel_size=7,
                               stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm1d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool1d(kernel_size=3, stride=2, padding=1)
        
        # Residual layers
        self.layer1 = self._make_layer(in_channels=64, out_channels=64, blocks=2, stride=1)
        self.layer2 = self._make_layer(in_channels=64, out_channels=128, blocks=2, stride=2)
        self.layer3 = self._make_layer(in_channels=128, out_channels=256, blocks=2, stride=2)
        self.layer4 = self._make_layer(in_channels=256, out_channels=512, blocks=2, stride=2)
        
        # Global average pooling to produce a fixed-length vector
        self.avgpool = nn.AdaptiveAvgPool1d(1)
        # Bottleneck fully-connected layer (optional, here keeping feature dim at 512)
        self.fc_bottleneck = nn.Linear(512, 512)
        
    def _make_layer(self, in_channels, out_channels, blocks, stride):
        """
        Create a sequential layer composed of multiple residual blocks.
        """
        downsample = None
        if stride != 1 or in_channels != out_channels:
            downsample = nn.Sequential(
                nn.Conv1d(in_channels, out_channels, kernel_size=1,
                          stride=stride, bias=False),
                nn.BatchNorm1d(out_channels)
            )
        layers = []
        layers.append(ResidualBlock1D(in_channels, out_channels, stride, downsample))
        for _ in range(1, blocks):
            layers.append(ResidualBlock1D(out_channels, out_channels))
        return nn.Sequential(*layers)
    
    def forward(self, x):
        # Input x: (batch_size, channels=2, length)
        x = self.conv1(x)    # -> (B, 64, L/2)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)  # -> (B, 64, L/4)
        
        x = self.layer1(x)   # -> (B, 64, ?)
        x = self.layer2(x)   # -> (B, 128, ?)
        x = self.layer3(x)   # -> (B, 256, ?)
        x = self.layer4(x)   # -> (B, 512, ?)
        
        # Global average pooling
        x = self.avgpool(x)  # -> (B, 512, 1)
        x = x.squeeze(-1)    # -> (B, 512)
        # Bottleneck transformation
        features = self.fc_bottleneck(x)  # (B, 512)
        return features

class DANN_LP(nn.Module):
    """
    Label predictor network that maps 512–dim features to the desired output classes.
    """
    def __init__(self, output_dim=7):
        super(DANN_LP, self).__init__()
        self.fc1 = nn.Linear(512, 256)
        self.bn_fc1 = nn.BatchNorm1d(256)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(256, output_dim)
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.bn_fc1(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

class DANN_DC(nn.Module):
    """
    Domain classifier network for DANN.
    
    This network applies a gradient reversal layer (using ReverseLayerF) to the feature vector
    before classifying it as either source or target.
    """
    def __init__(self):
        super(DANN_DC, self).__init__()
        self.fc1 = nn.Linear(512, 100)
        self.fc2 = nn.Linear(100, 2)
        
    def forward(self, x, alpha):
        # Reverse gradient during the backward pass
        x = GradReverse.apply(x, alpha)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

#%% CORAL
def coral_loss(source, target):
    """
    Computes the CORAL loss between two matrices.
    Assumes input tensors are of shape (batch_size, feature_dim).
    """
    d = source.size(1)
    ns = source.size(0)
    nt = target.size(0)
    # Center the features
    source_mean = torch.mean(source, dim=0, keepdim=True)
    target_mean = torch.mean(target, dim=0, keepdim=True)
    source_centered = source - source_mean
    target_centered = target - target_mean
    # Compute covariance matrices
    cov_source = (source_centered.t() @ source_centered) / (ns - 1)
    cov_target = (target_centered.t() @ target_centered) / (nt - 1)
    # Frobenius norm between covariance matrices (scaled)
    loss = torch.mean((cov_source - cov_target) ** 2)
    loss = loss / (4 * d * d)
    return loss
        
class CORAL_G(nn.Module):
    """
    Deep ResNet generator that extracts features at multiple depths for CORAL.
    
    Architecture:
      - An initial convolution + batchnorm + ReLU + maxpool.
      - Four layers (residual blocks) built with 1D convolutions.
      - Intermediate features are extracted after layer1 (early), layer2 (middle),
        and layer3 (late) via global average pooling.
      - The final layer (layer4) is pooled and passed through a fully-connected
        bottleneck to obtain a 512–dim feature for classification.
    """
    def __init__(self):
        super(CORAL_G, self).__init__()
        # Initial convolution layer
        self.conv1 = nn.Conv1d(in_channels=2, out_channels=64, kernel_size=7,
                               stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm1d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool1d(kernel_size=3, stride=2, padding=1)
        
        # Residual layers
        self.layer1 = self._make_layer(in_channels=64, out_channels=64, blocks=2, stride=1)
        self.layer2 = self._make_layer(in_channels=64, out_channels=128, blocks=2, stride=2)
        self.layer3 = self._make_layer(in_channels=128, out_channels=256, blocks=2, stride=2)
        self.layer4 = self._make_layer(in_channels=256, out_channels=512, blocks=2, stride=2)
        
        # Global average pooling (adaptive to any sequence length)
        self.avgpool = nn.AdaptiveAvgPool1d(1)
        # Bottleneck fully-connected layer mapping 512->512 for classification.
        self.fc_bottleneck = nn.Linear(512, 512)
        
    def _make_layer(self, in_channels, out_channels, blocks, stride):
        """
        Creates a sequential layer composed of multiple residual blocks.
        If the stride is not 1 or the channel dimensions differ,
        a downsampling layer is used.
        """
        downsample = None
        if stride != 1 or in_channels != out_channels:
            downsample = nn.Sequential(
                nn.Conv1d(in_channels, out_channels, kernel_size=1,
                          stride=stride, bias=False),
                nn.BatchNorm1d(out_channels)
            )
        layers = []
        layers.append(ResidualBlock1D(in_channels, out_channels, stride, downsample))
        for _ in range(1, blocks):
            layers.append(ResidualBlock1D(out_channels, out_channels))
        return nn.Sequential(*layers)
    
    def forward(self, x):
        # Input x: (batch_size, channels=2, length)
        x = self.conv1(x)   # (B, 64, L/2)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x) # (B, 64, L/4)
        
        # Layer 1: Early features
        x1 = self.layer1(x)           # (B, 64, L1)
        early = self.avgpool(x1)      # (B, 64, 1)
        early = early.squeeze(-1)     # (B, 64)
        
        # Layer 2: Middle features
        x2 = self.layer2(x1)          # (B, 128, L2)
        middle = self.avgpool(x2)     # (B, 128, 1)
        middle = middle.squeeze(-1)   # (B, 128)
        
        # Layer 3: Late features for CORAL loss
        x3 = self.layer3(x2)          # (B, 256, L3)
        late = self.avgpool(x3)       # (B, 256, 1)
        late = late.squeeze(-1)       # (B, 256)
        
        # Layer 4: Final block for classification
        x4 = self.layer4(x3)          # (B, 512, L4)
        pooled = self.avgpool(x4)     # (B, 512, 1)
        pooled = pooled.squeeze(-1)   # (B, 512)
        final = self.fc_bottleneck(pooled)  # (B, 512)
        
        # Return multi-level features for deep CORAL
        return early, middle, late, final

class CORAL_C(nn.Module):
    """
    Classifier network that maps the 512–dim bottleneck features to the output classes.
    
    Architecture:
      - A fully-connected layer (512 -> 256) with batch normalization, ReLU, and dropout.
      - A final fully-connected layer mapping 256 -> output_dim.
    """
    def __init__(self, output_dim):
        super(CORAL_C, self).__init__()
        self.fc1 = nn.Linear(512, 256)
        self.bn_fc1 = nn.BatchNorm1d(256)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(256, output_dim)
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.bn_fc1(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x


class DeepCORAL:
    def __init__(self, G, C, device, 
                 S_train_loader, S_val_loader, 
                 T_train_loader, T_val_loader,
                 class_subset, n_classes, lr, n_epochs, n_runs, patience, 
                 lambda_coral=0.5, deep_weights=(1.0, 1.0, 1.0)):
        """
        deep_weights: tuple of weights for the CORAL loss at (early, middle, late) layers.
        """
        self.G = G
        self.C = C
        self.device = device
        self.S_train_loader = S_train_loader
        self.S_val_loader = S_val_loader
        self.T_train_loader = T_train_loader
        self.T_val_loader = T_val_loader
        self.class_subset = class_subset
        self.n_classes = n_classes
        self.lr = lr
        self.n_epochs = n_epochs
        self.n_runs = n_runs
        self.patience = patience
        self.lambda_coral = lambda_coral
        self.deep_weights = deep_weights
        
    def evaluate(self, netG, netC, loader):
        netG.eval()
        netC.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for data, labels in loader:
                data, labels = data.to(self.device), labels.to(self.device)
                # Only need the classification branch (late_fc)
                _, _, _, cls_feat = netG(data)
                outputs = netC(cls_feat)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        return correct / total

    def run(self):
        best_s_acc_list = []
        best_t_acc_list = []
        for run in range(self.n_runs):
            print(f"Deep CORAL Run {run+1}/{self.n_runs}")
            # Instantiate new networks for each run
            netG = self.G().to(self.device)
            netC = self.C(self.n_classes).to(self.device)
            optimizer = torch.optim.Adam(list(netG.parameters()) + list(netC.parameters()), lr=self.lr)
            criterion = nn.CrossEntropyLoss()
            best_val_acc = 0.0
            patience_counter = 0
            
            for epoch in range(self.n_epochs):
                netG.train()
                netC.train()
                # Use zip to iterate over source and target batches in parallel.
                for (s_data, s_labels), (t_data, _) in zip(self.S_train_loader, self.T_train_loader):
                    s_data, s_labels = s_data.to(self.device), s_labels.to(self.device)
                    t_data = t_data.to(self.device)
                    
                    # Forward pass for source: get all features
                    s_early, s_middle, s_late, s_cls = netG(s_data)
                    # Classification loss on source (using the classification feature)
                    loss_cls = criterion(netC(s_cls), s_labels)
                    
                    # Forward pass for target: we only need the intermediate features
                    t_early, t_middle, t_late, _ = netG(t_data)
                    
                    # Compute CORAL loss for each level
                    loss_early = coral_loss(s_early, t_early)
                    loss_middle = coral_loss(s_middle, t_middle)
                    loss_late = coral_loss(s_late, t_late)
                    loss_coral_total = (self.deep_weights[0] * loss_early +
                                        self.deep_weights[1] * loss_middle +
                                        self.deep_weights[2] * loss_late)
                    
                    total_loss = loss_cls + self.lambda_coral * loss_coral_total
                    
                    optimizer.zero_grad()
                    total_loss.backward()
                    optimizer.step()
                    
                # End-of-epoch evaluation on source validation set
                s_acc = self.evaluate(netG, netC, self.S_val_loader)
                t_acc = self.evaluate(netG, netC, self.T_val_loader)
                print(f"Epoch {epoch+1}: Source Val Acc = {s_acc:.4f}, Target Val Acc = {t_acc:.4f}")
                
                # Early stopping on source validation accuracy
                if s_acc > best_val_acc:
                    best_val_acc = s_acc
                    best_model_G = netG.state_dict()
                    best_model_C = netC.state_dict()
                    patience_counter = 0
                else:
                    patience_counter += 1
                if patience_counter >= self.patience:
                    print("Early stopping triggered.")
                    break
                    
            # Load best model from this run
            netG.load_state_dict(best_model_G)
            netC.load_state_dict(best_model_C)
            s_acc = self.evaluate(netG, netC, self.S_val_loader)
            t_acc = self.evaluate(netG, netC, self.T_val_loader)
            print(f"Run {run+1} finished: Best Source Val Acc = {s_acc:.4f}, Target Val Acc = {t_acc:.4f}\n")
            best_s_acc_list.append(s_acc)
            best_t_acc_list.append(t_acc)
            
        avg_s_acc = np.mean(best_s_acc_list)
        avg_t_acc = np.mean(best_t_acc_list)
        print(f"Deep CORAL: Average Source Val Acc = {avg_s_acc:.4f}, Average Target Val Acc = {avg_t_acc:.4f}")
        return avg_s_acc, avg_t_acc

#%% STAR
class STAR_G(nn.Module):
    """
    Deep ResNet feature extractor for STAR.
    
    This network accepts a 2–channel 1D signal and extracts a 512–dimensional
    feature vector via several residual layers.
    """
    def __init__(self):
        super(STAR_G, self).__init__()
        # Initial convolutional block
        self.conv1 = nn.Conv1d(in_channels=2, out_channels=64, kernel_size=7,
                               stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm1d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool1d(kernel_size=3, stride=2, padding=1)
        
        # Residual layers
        self.layer1 = self._make_layer(64, 64, blocks=2, stride=1)
        self.layer2 = self._make_layer(64, 128, blocks=2, stride=2)
        self.layer3 = self._make_layer(128, 256, blocks=2, stride=2)
        self.layer4 = self._make_layer(256, 512, blocks=2, stride=2)
        
        # Global average pooling to obtain a fixed-length feature vector
        self.avgpool = nn.AdaptiveAvgPool1d(1)
        # Optional bottleneck fully-connected layer (maps 512->512)
        self.fc_bottleneck = nn.Linear(512, 512)
        
    def _make_layer(self, in_channels, out_channels, blocks, stride):
        downsample = None
        if stride != 1 or in_channels != out_channels:
            downsample = nn.Sequential(
                nn.Conv1d(in_channels, out_channels, kernel_size=1,
                          stride=stride, bias=False),
                nn.BatchNorm1d(out_channels)
            )
        layers = []
        layers.append(ResidualBlock1D(in_channels, out_channels, stride, downsample))
        for _ in range(1, blocks):
            layers.append(ResidualBlock1D(out_channels, out_channels))
        return nn.Sequential(*layers)
    
    def forward(self, x):
        # x: (batch_size, channels=2, length)
        x = self.conv1(x)      # (B, 64, L/2)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)    # (B, 64, L/4)
        
        x = self.layer1(x)     # (B, 64, ?)
        x = self.layer2(x)     # (B, 128, ?)
        x = self.layer3(x)     # (B, 256, ?)
        x = self.layer4(x)     # (B, 512, ?)
        
        x = self.avgpool(x)    # (B, 512, 1)
        x = x.squeeze(-1)      # (B, 512)
        x = self.fc_bottleneck(x)  # (B, 512)
        return x

class STAR_C(nn.Module):
    """
    Stochastic classifier network for STAR.
    
    This network receives the 512–dim features from DeepResNet_STAR_G,
    applies a fully connected layer with batch normalization and ReLU,
    and then uses a learned weight distribution (mu2, sigma2) to sample
    classifier weights. During training, it samples num_classifiers_train classifiers,
    while during evaluation it can either use only the mean (only_mu=True) or
    sample num_classifiers_test classifiers.
    """
    def __init__(self, output_dim, num_classifiers_train=2, num_classifiers_test=20,
                 init='kaiming_u', use_init=False):
        super(STAR_C, self).__init__()
        self.num_classifiers_train = num_classifiers_train
        self.num_classifiers_test = num_classifiers_test
        self.init = init

        function_init = {
            'kaiming_u': nn.init.kaiming_uniform_,
            'kaiming_n': nn.init.kaiming_normal_,
            'xavier': nn.init.xavier_normal_
        }

        # Change input dimension to 512 (from the DeepResNet feature extractor)
        self.fc1 = nn.Linear(512, 128)
        self.bn1_fc = nn.BatchNorm1d(128)

        # Learnable parameters for the classifier weight distribution
        self.mu2 = nn.Parameter(torch.randn(output_dim, 128))
        self.sigma2 = nn.Parameter(torch.zeros(output_dim, 128))

        if use_init:
            for item in [self.mu2, self.sigma2]:
                function_init[self.init](item)

        self.b2 = nn.Parameter(torch.zeros(output_dim))

    def forward(self, x, only_mu=True):
        x = self.fc1(x)
        x = F.relu(self.bn1_fc(x))

        sigma2_pos = torch.sigmoid(self.sigma2)
        fc2_distribution = torch.distributions.Normal(self.mu2, sigma2_pos)

        if self.training:
            classifiers = []
            for _ in range(self.num_classifiers_train):
                fc2_w = fc2_distribution.rsample()
                classifiers.append([fc2_w, self.b2])

            outputs = []
            for classifier in classifiers:
                out = F.linear(x, classifier[0], classifier[1])
                outputs.append(out)
            return outputs
        else:
            if only_mu:
                out = F.linear(x, self.mu2, self.b2)
                return [out]
            else:
                classifiers = []
                for _ in range(self.num_classifiers_test):
                    fc2_w = fc2_distribution.rsample()
                    classifiers.append([fc2_w, self.b2])
                outputs = []
                for classifier in classifiers:
                    out = F.linear(x, classifier[0], classifier[1])
                    outputs.append(out)
                return outputs

#%% MCD
class MCD_G(nn.Module):
    def __init__(self):
        super(MCD_G, self).__init__()
        # Initial convolutional block
        self.conv1 = nn.Conv1d(in_channels=2, out_channels=64, kernel_size=7,
                               stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm1d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool1d(kernel_size=3, stride=2, padding=1)
        
        # Residual layers
        self.layer1 = self._make_layer(in_channels=64, out_channels=64, blocks=2, stride=1)
        self.layer2 = self._make_layer(in_channels=64, out_channels=128, blocks=2, stride=2)
        self.layer3 = self._make_layer(in_channels=128, out_channels=256, blocks=2, stride=2)
        self.layer4 = self._make_layer(in_channels=256, out_channels=512, blocks=2, stride=2)
        
        # Global average pooling to get a fixed-length feature vector
        self.avgpool = nn.AdaptiveAvgPool1d(1)
        # Bottleneck fully-connected layer mapping 512 -> 512 (optional)
        self.fc_bottleneck = nn.Linear(512, 512)
        
    def _make_layer(self, in_channels, out_channels, blocks, stride):
        downsample = None
        if stride != 1 or in_channels != out_channels:
            downsample = nn.Sequential(
                nn.Conv1d(in_channels, out_channels, kernel_size=1,
                          stride=stride, bias=False),
                nn.BatchNorm1d(out_channels)
            )
        layers = []
        layers.append(ResidualBlock1D(in_channels, out_channels, stride, downsample))
        for _ in range(1, blocks):
            layers.append(ResidualBlock1D(out_channels, out_channels))
        return nn.Sequential(*layers)
    
    def forward(self, x):
        # Input x shape: (batch_size, 2, length)
        x = self.conv1(x)      # -> (B, 64, L/2)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)    # -> (B, 64, L/4)
        
        x = self.layer1(x)     # -> (B, 64, ?)
        x = self.layer2(x)     # -> (B, 128, ?)
        x = self.layer3(x)     # -> (B, 256, ?)
        x = self.layer4(x)     # -> (B, 512, ?)
        
        x = self.avgpool(x)    # -> (B, 512, 1)
        x = x.squeeze(-1)      # -> (B, 512)
        x = self.fc_bottleneck(x)  # -> (B, 512)
        return x

class MCD_C(nn.Module):
    def __init__(self, output_dim):
        super(MCD_C, self).__init__()
        # Since our feature extractor now outputs a 512-dim vector,
        # adjust the input dimension accordingly.
        self.fc1 = nn.Linear(512, 128)
        self.fc2 = nn.Linear(128, output_dim)
    
    def forward(self, x, reverse=False, lambda_=1.0):
        # Optionally apply gradient reversal
        if reverse:
            x = grad_reverse(x, lambda_)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

#%% JAN
class JAN_G(nn.Module):
    def __init__(self):
        super(JAN_G, self).__init__()
        # Initial convolutional block
        self.conv1 = nn.Conv1d(in_channels=2, out_channels=64, kernel_size=7,
                               stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm1d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool1d(kernel_size=3, stride=2, padding=1)
        
        # Residual layers
        self.layer1 = self._make_layer(64, 64, blocks=2, stride=1)
        self.layer2 = self._make_layer(64, 128, blocks=2, stride=2)
        self.layer3 = self._make_layer(128, 256, blocks=2, stride=2)
        self.layer4 = self._make_layer(256, 512, blocks=2, stride=2)
        
        # Global average pooling to obtain a fixed-length feature vector
        self.avgpool = nn.AdaptiveAvgPool1d(1)
        # Bottleneck layer mapping 512 -> 512 dimensions
        self.fc_bottleneck = nn.Linear(512, 512)
    
    def _make_layer(self, in_channels, out_channels, blocks, stride):
        downsample = None
        if stride != 1 or in_channels != out_channels:
            downsample = nn.Sequential(
                nn.Conv1d(in_channels, out_channels, kernel_size=1,
                          stride=stride, bias=False),
                nn.BatchNorm1d(out_channels)
            )
        layers = []
        layers.append(ResidualBlock1D(in_channels, out_channels, stride, downsample))
        for _ in range(1, blocks):
            layers.append(ResidualBlock1D(out_channels, out_channels))
        return nn.Sequential(*layers)
    
    def forward(self, x):
        # x: (batch_size, 2, length)
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        x = self.avgpool(x)  # shape: (batch, 512, 1)
        x = x.squeeze(-1)    # shape: (batch, 512)
        x = self.fc_bottleneck(x)  # shape: (batch, 512)
        
        # Debug: Ensure x is a valid tensor
        if x is None:
            raise ValueError("Generator output is None. Check your forward pass.")
        # You can also uncomment the next line to print the shape during debugging.
        # print("Generator output shape:", x.shape)
        return x

class C_JAN(nn.Module):
    def __init__(self, output_dim):
        super(C_JAN, self).__init__()
        self.fc1 = nn.Linear(512, 128)
        self.fc2 = nn.Linear(128, output_dim)
        
    def forward(self, x, return_intermediate=False):
        inter = torch.relu(self.fc1(x))
        out = self.fc2(inter)
        if return_intermediate:
            return out, inter
        else:
            return out

In [2]:
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import DataLoader, TensorDataset, Subset

def create_cv_loaders(X_path, Y_path, batch_size=64, n_splits=5, permute=True):
    """
    Create cross-validated dataloaders with stratified splits.

    Parameters
    ----------
    X_path : Path for feature set or an ndarray
    Y_path : Path for label set or an ndarray
    batch_size : Batch size for the dataloaders
    n_splits : Number of cross validation folds (e.g., 5 or 10)
    permute : Whether to permute dimensions of the input tensor 
              (e.g., to change (N, 2, 4096) to (N, 4096, 2))

    Returns
    -------
    cv_loaders : List of tuples (train_loader, val_loader) for each fold
    """
    # Load the data from file or use provided arrays
    if isinstance(X_path, np.ndarray):
        X = X_path
    else:
        X = np.load(X_path)
        
    if isinstance(Y_path, np.ndarray):
        Y = Y_path
    else:
        Y = np.load(Y_path)
    
    # Convert arrays to tensors
    X_tensor = torch.tensor(X, dtype=torch.float32).to(device)
    Y_tensor = torch.tensor(Y, dtype=torch.long).to(device)
    
    if permute:
        X_tensor = X_tensor.permute(0, 2, 1)
    
    # Create a TensorDataset
    dataset = TensorDataset(X_tensor, Y_tensor)
    
    # Initialize StratifiedKFold
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    
    cv_loaders = []
    
    # The first parameter of skf.split can be a dummy array as the split is based on y_labels
    for train_idx, val_idx in skf.split(np.zeros(len(Y)), Y):
        train_subset = Subset(dataset, train_idx)
        val_subset = Subset(dataset, val_idx)
        
        train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False)
        
        cv_loaders.append((train_loader, val_loader))
    
    return cv_loaders





In [ ]:
#%% Cross-validation
# Load testbed data
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
file_path = "/home/ash/ic3/testbed_da/data"

# Classes
class_subset = ["bpsk", "qpsk", "16qam", "8apsk"]

X = np.load(file_path + "/sim_X.npy")
Y = np.load(file_path + "/sim_Y.npy")

sou_snr = 22
tar_snr = 10

t_deep_coral_acc = []
t_base_acc = []
t_dann_acc = []
t_mcd_acc = []
t_star_acc = []
t_jan_acc = []
t_golden_acc = []

n_runs = 3
n_epochs= 50
lr = 0.001
n_snr= 3

# In-domain case with cross validation
for i in range(n_snr):
    source_mask = (Y[:, 1] == tar_snr)
    X_g = X[source_mask]
    Y_g = Y[source_mask]
    Y_g = Y_g[:, 0]
    
    cv_loaders = create_cv_loaders(X_g, Y_g, batch_size=128, n_splits=5, permute=False)
    
    fold_results = []
    for fold, (S_train_loader, S_val_loader) in enumerate(cv_loaders, start=1):
        print(f'In-domain - Fold {fold}')
        _, t_golden = base.Base(
            model_cls=DeepResNet,
            device=device,
            S_train_loader=S_train_loader, 
            S_val_loader=S_val_loader,
            T_val_loader=S_val_loader,
            class_subset=class_subset, 
            n_classes=len(class_subset),
            lr=lr,
            n_epochs=n_epochs,
            n_runs=n_runs
        ).run()
        torch.cuda.empty_cache()
        fold_results.append(t_golden)
    
    avg_accuracy = sum(fold_results) / len(fold_results)
    t_golden_acc.append(avg_accuracy)
    
    tar_snr += 4

tar_snr = 10
for i in range(n_snr):
    source_mask = (Y[:, 1] == sou_snr)
    target_mask = (Y[:, 1] == tar_snr)
    
    X_s = X[source_mask]
    Y_s = Y[source_mask]
    Y_s = Y_s[:, 0]
    
    X_t = X[target_mask]
    Y_t = Y[target_mask]
    Y_t = Y_t[:, 0]
    
    cv_source_loaders = create_cv_loaders(X_s, Y_s, batch_size=128, n_splits=5, permute=False)
    cv_target_loaders = create_cv_loaders(X_t, Y_t, batch_size=128, n_splits=5, permute=False)
    
    fold_base_results = []
    fold_dann_results = []
    fold_deep_results = []
    fold_star_results = []
    fold_mcd_results = []
    fold_jan_results = []
    
    for fold, ((S_train_loader, S_val_loader), (T_train_loader, T_val_loader)) in enumerate(
            zip(cv_source_loaders, cv_target_loaders), start=1):
        
        print(f'Base - Fold {fold}')
        _, t_base = base.Base(
            model_cls=DeepResNet,
            device=device,
            S_train_loader=S_train_loader, 
            S_val_loader=S_val_loader,
            T_val_loader=T_val_loader,
            class_subset=class_subset, 
            n_classes=len(class_subset),
            lr=lr,
            n_epochs=n_epochs,
            n_runs=n_runs
        ).run()
        torch.cuda.empty_cache()
        fold_base_results.append(t_base)
        
        print(f'DANN - Fold {fold}')
        _, t_dann = dann.DAN(
            dann.DANN,
            FA=DANN_F,
            LP=DANN_LP,
            DC=DANN_DC,
            device=device,
            S_train_loader=S_train_loader,
            S_val_loader=S_val_loader,
            T_train_loader=T_train_loader,
            T_val_loader=T_val_loader,
            class_subset=class_subset,
            n_classes=len(class_subset),
            lr=lr,
            n_epochs=n_epochs,
            n_runs=n_runs
        ).run()
        torch.cuda.empty_cache()
        fold_dann_results.append(t_dann)
        
        print(f'Deep CORAL - Fold {fold}')
        _, t_deep = DeepCORAL(
            G=CORAL_G, 
            C=CORAL_C, 
            device=device, 
            S_train_loader=S_train_loader,
            S_val_loader=S_val_loader,
            T_train_loader=T_train_loader,
            T_val_loader=T_val_loader,
            class_subset=class_subset,
            n_classes=len(class_subset),
            lr=lr, 
            n_epochs=n_epochs, 
            n_runs=n_runs,
            patience=5,
            lambda_coral=0.5,
            deep_weights=(1.0, 1.0, 1.0)
        ).run()
        torch.cuda.empty_cache()
        fold_deep_results.append(t_deep)
        
        print(f'STAR - Fold {fold}')
        _, t_star = star.Star(
            G=STAR_G,
            C=STAR_C,
            device=device,
            S_train_loader=S_train_loader,
            S_val_loader=S_val_loader,  
            T_train_loader=T_train_loader,
            T_val_loader=T_val_loader,
            class_subset=class_subset,
            n_classes=len(class_subset),
            lr=lr,
            n_epochs=n_epochs,
            n_runs=n_runs,
            patience=5
        ).run()
        torch.cuda.empty_cache()
        fold_star_results.append(t_star)
        
        print(f'MCD - Fold {fold}')
        _, t_mcd = mcd.Mcd(
            G=MCD_G,
            C=MCD_C,
            device=device,
            S_train_loader=S_train_loader,
            S_val_loader=S_val_loader,  
            T_train_loader=T_train_loader,
            T_val_loader=T_val_loader,
            class_subset=class_subset,
            n_classes=len(class_subset),
            lr=lr,
            n_epochs=n_epochs,
            n_runs=n_runs,
            patience=5
        ).run()
        torch.cuda.empty_cache()
        fold_mcd_results.append(t_mcd)
        
        print(f'JAN - Fold {fold}')
        _, t_jan = jan.Jan(
            C=C_JAN,
            G=JAN_G,
            num_classes=len(class_subset),
            device=device,
            S_train_loader=S_train_loader,
            T_train_loader=T_train_loader,
            S_val_loader=S_val_loader,
            T_val_loader=T_val_loader,
            n_epochs=n_epochs,
            lr=lr,
            lambda_jmmd=0.1,
            n_runs=n_runs
        ).run()
        torch.cuda.empty_cache()
        fold_jan_results.append(t_jan)
    
    avg_base = sum(fold_base_results) / len(fold_base_results)
    avg_dann = sum(fold_dann_results) / len(fold_dann_results)
    avg_deep = sum(fold_deep_results) / len(fold_deep_results)
    avg_star = sum(fold_star_results) / len(fold_star_results)
    avg_mcd  = sum(fold_mcd_results) / len(fold_mcd_results)
    avg_jan  = sum(fold_jan_results) / len(fold_jan_results)
    
    t_base_acc.append(avg_base)
    t_dann_acc.append(avg_dann)
    t_deep_coral_acc.append(avg_deep)
    t_star_acc.append(avg_star)
    t_mcd_acc.append(avg_mcd)
    t_jan_acc.append(avg_jan)
    
    tar_snr += 4 


In-domain - Fold 1

Run 1/3
Epoch 1/50, Train Loss: 0.2799, Train Acc: 0.9075, Val Loss: 1.0979, Val Acc: 0.6769
Epoch 2/50, Train Loss: 0.0450, Train Acc: 0.9862, Val Loss: 0.1120, Val Acc: 0.9706
Epoch 3/50, Train Loss: 0.0237, Train Acc: 0.9933, Val Loss: 0.9304, Val Acc: 0.7782
Epoch 4/50, Train Loss: 0.0206, Train Acc: 0.9940, Val Loss: 0.0361, Val Acc: 0.9862
Epoch 5/50, Train Loss: 0.0186, Train Acc: 0.9954, Val Loss: 0.2881, Val Acc: 0.9125
Epoch 6/50, Train Loss: 0.0194, Train Acc: 0.9939, Val Loss: 0.0373, Val Acc: 0.9862
Epoch 7/50, Train Loss: 0.0091, Train Acc: 0.9981, Val Loss: 0.0024, Val Acc: 1.0000
Epoch 8/50, Train Loss: 0.0038, Train Acc: 0.9994, Val Loss: 0.0534, Val Acc: 0.9790
Epoch 9/50, Train Loss: 0.0080, Train Acc: 0.9979, Val Loss: 1.0557, Val Acc: 0.7122
Epoch 10/50, Train Loss: 0.0051, Train Acc: 0.9987, Val Loss: 0.0033, Val Acc: 0.9994
Epoch 11/50, Train Loss: 0.0050, Train Acc: 0.9991, Val Loss: 0.0017, Val Acc: 0.9994
Epoch 12/50, Train Loss: 0.0058, Tr

/home/ash/.conda/envs/torch/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch 1/50, Train Loss: 0.4242, Train Acc: 0.8331, Val Loss: 1.0558, Val Acc: 0.7458
Epoch 2/50, Train Loss: 0.1264, Train Acc: 0.9522, Val Loss: 0.7837, Val Acc: 0.7848
Epoch 3/50, Train Loss: 0.0672, Train Acc: 0.9762, Val Loss: 0.7755, Val Acc: 0.8112
Epoch 4/50, Train Loss: 0.0355, Train Acc: 0.9877, Val Loss: 0.1726, Val Acc: 0.9478
Epoch 5/50, Train Loss: 0.0307, Train Acc: 0.9895, Val Loss: 0.1793, Val Acc: 0.9317
Epoch 6/50, Train Loss: 0.0110, Train Acc: 0.9973, Val Loss: 0.1789, Val Acc: 0.9394
Epoch 7/50, Train Loss: 0.0124, Train Acc: 0.9954, Val Loss: 0.0257, Val Acc: 0.9946
Epoch 8/50, Train Loss: 0.0053, Train Acc: 0.9985, Val Loss: 0.3472, Val Acc: 0.9191
Epoch 9/50, Train Loss: 0.0099, Train Acc: 0.9966, Val Loss: 0.1009, Val Acc: 0.9640
Epoch 10/50, Train Loss: 0.0036, Train Acc: 0.9990, Val Loss: 0.0086, Val Acc: 0.9976
Epoch 11/50, Train Loss: 0.0024, Train Acc: 0.9996, Val Loss: 0.0094, Val Acc: 0.9970
Epoch 12/50, Train Loss: 0.0007, Train Acc: 1.0000, Val Loss: 0

/home/ash/.conda/envs/torch/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch 1/50, Train Loss: 0.3893, Train Acc: 0.8460, Val Loss: 1.6280, Val Acc: 0.7464
Epoch 2/50, Train Loss: 0.1185, Train Acc: 0.9547, Val Loss: 0.5719, Val Acc: 0.8094
Epoch 3/50, Train Loss: 0.0552, Train Acc: 0.9810, Val Loss: 8.8329, Val Acc: 0.5288
Epoch 4/50, Train Loss: 0.0285, Train Acc: 0.9901, Val Loss: 0.1301, Val Acc: 0.9490
Epoch 5/50, Train Loss: 0.0145, Train Acc: 0.9958, Val Loss: 0.0755, Val Acc: 0.9754
Epoch 6/50, Train Loss: 0.0050, Train Acc: 0.9990, Val Loss: 0.1450, Val Acc: 0.9496
Epoch 7/50, Train Loss: 0.0048, Train Acc: 0.9990, Val Loss: 0.2481, Val Acc: 0.9173
Epoch 8/50, Train Loss: 0.0051, Train Acc: 0.9985, Val Loss: 0.0094, Val Acc: 0.9970
Epoch 9/50, Train Loss: 0.0023, Train Acc: 0.9993, Val Loss: 0.6923, Val Acc: 0.8411
Epoch 10/50, Train Loss: 0.0009, Train Acc: 0.9999, Val Loss: 0.0122, Val Acc: 0.9958
Epoch 11/50, Train Loss: 0.0004, Train Acc: 1.0000, Val Loss: 0.0016, Val Acc: 0.9994
Epoch 12/50, Train Loss: 0.0003, Train Acc: 1.0000, Val Loss: 0

...

Epoch 1/50, Train Loss: 0.3997, Train Acc: 0.8387, Val Loss: 3.9121, Val Acc: 0.6809
Epoch 2/50, Train Loss: 0.1226, Train Acc: 0.9555, Val Loss: 0.1131, Val Acc: 0.9568
Epoch 3/50, Train Loss: 0.0471, Train Acc: 0.9831, Val Loss: 1.6134, Val Acc: 0.6155
Epoch 4/50, Train Loss: 0.0325, Train Acc: 0.9885, Val Loss: 1.0162, Val Acc: 0.7570
Epoch 5/50, Train Loss: 0.0161, Train Acc: 0.9948, Val Loss: 0.7169, Val Acc: 0.8380
Epoch 6/50, Train Loss: 0.0067, Train Acc: 0.9985, Val Loss: 0.0495, Val Acc: 0.9772
Epoch 7/50, Train Loss: 0.0051, Train Acc: 0.9985, Val Loss: 0.0278, Val Acc: 0.9916
Epoch 8/50, Train Loss: 0.0028, Train Acc: 0.9994, Val Loss: 0.0937, Val Acc: 0.9670
Epoch 9/50, Train Loss: 0.0020, Train Acc: 0.9994, Val Loss: 0.0006, Val Acc: 1.0000
Epoch 10/50, Train Loss: 0.0044, Train Acc: 0.9987, Val Loss: 0.0005, Val Acc: 1.0000
Epoch 11/50, Train Loss: 0.0005, Train Acc: 1.0000, Val Loss: 0.0009, Val Acc: 1.0000
Epoch 12/50, Train Loss: 0.0005, Train Acc: 1.0000, Val Loss: 0

/home/ash/.conda/envs/torch/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch 1/50, Loss: 2.1529, Domain Loss: 1.2236, Class Loss: 0.9293
Epoch 2/50, Loss: 1.7939, Domain Loss: 1.3597, Class Loss: 0.4342
Epoch 3/50, Loss: 4.7291, Domain Loss: 4.3395, Class Loss: 0.3896
Epoch 4/50, Loss: 5.2712, Domain Loss: 4.8652, Class Loss: 0.4060
Epoch 5/50, Loss: 2.5768, Domain Loss: 2.2356, Class Loss: 0.3412
Epoch 6/50, Loss: 4.9091, Domain Loss: 4.5740, Class Loss: 0.3351
Epoch 7/50, Loss: 7.6812, Domain Loss: 7.3244, Class Loss: 0.3569
Epoch 8/50, Loss: 7.9179, Domain Loss: 7.5693, Class Loss: 0.3486
Epoch 9/50, Loss: 4.7627, Domain Loss: 4.4032, Class Loss: 0.3595
Epoch 10/50, Loss: 3.2616, Domain Loss: 2.8976, Class Loss: 0.3640
Epoch 11/50, Loss: 3.8321, Domain Loss: 3.4755, Class Loss: 0.3566
Epoch 12/50, Loss: 3.2483, Domain Loss: 2.9513, Class Loss: 0.2970
Epoch 13/50, Loss: 3.4150, Domain Loss: 3.1775, Class Loss: 0.2375
Epoch 14/50, Loss: 3.7523, Domain Loss: 3.4352, Class Loss: 0.3171
Epoch 15/50, Loss: 2.7021, Domain Loss: 2.2502, Class Loss: 0.4518
Epoc

/home/ash/.conda/envs/torch/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch 1/50, Train Loss: 0.4091, Train Acc: 0.8262, Val Loss: 2.3343, Val Acc: 0.7464
Epoch 2/50, Train Loss: 0.1157, Train Acc: 0.9594, Val Loss: 0.3920, Val Acc: 0.8633
Epoch 3/50, Train Loss: 0.0542, Train Acc: 0.9813, Val Loss: 0.1735, Val Acc: 0.9388
Epoch 4/50, Train Loss: 0.0396, Train Acc: 0.9862, Val Loss: 0.2563, Val Acc: 0.9293
Epoch 5/50, Train Loss: 0.0195, Train Acc: 0.9939, Val Loss: 0.0626, Val Acc: 0.9736
Epoch 6/50, Train Loss: 0.0189, Train Acc: 0.9948, Val Loss: 0.4719, Val Acc: 0.8879
Epoch 7/50, Train Loss: 0.0035, Train Acc: 0.9994, Val Loss: 0.0875, Val Acc: 0.9802
Epoch 8/50, Train Loss: 0.0276, Train Acc: 0.9933, Val Loss: 0.0862, Val Acc: 0.9766
Epoch 9/50, Train Loss: 0.0086, Train Acc: 0.9985, Val Loss: 0.0432, Val Acc: 0.9826
Epoch 10/50, Train Loss: 0.0060, Train Acc: 0.9982, Val Loss: 0.0379, Val Acc: 0.9856
Epoch 11/50, Train Loss: 0.0017, Train Acc: 0.9996, Val Loss: 0.0006, Val Acc: 1.0000
Epoch 12/50, Train Loss: 0.0026, Train Acc: 0.9996, Val Loss: 0

In [ ]:
x = np.arange(1, n_snr+1)
print(len(t_deep_coral_acc))
print(len(x))

plt.figure(figsize=(10, 6))
plt.plot(x, t_base_acc, marker='o', linestyle='-', label='Base')
plt.plot(x, t_dann_acc, marker='^', linestyle='--', label='DANN')
plt.plot(x, t_star_acc, marker='<', linestyle='--', label='Star')
plt.plot(x, t_mcd_acc, marker='D', linestyle='--', label='MCD')
plt.plot(x, t_deep_coral_acc, marker='v', linestyle='--', label='DCORAL')
plt.plot(x, t_jan_acc, marker='x', linestyle='--', label='JAN')
plt.plot(x, t_golden_acc, marker='>', linestyle='-', label='In-domain')

plt.xlabel('SNR (dB)')
plt.ylabel('Acc (%)')
plt.legend()
plt.grid(True)
plt.title('Simulated Data SNR-to-SNR DA Comparison')
plt.show()

In [ ]:
#%% No Cross-validation
# Load testbed data
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
file_path = "/home/ash/ic3/testbed_da/data"

# Classes
class_subset = ["bpsk", "qpsk", "16qam", "8apsk"]

# Split source, target
# try selecting some of the mods, not all
X = np.load(file_path + "/sim_X.npy")
Y = np.load(file_path + "/sim_Y.npy")

sou_snr = 22
tar_snr = 10

t_deep_coral_acc = []
t_base_acc = []
t_dann_acc = []
t_mcd_acc = []
t_star_acc = []
t_jan_acc = []
t_golden_acc = []

n_runs = 10
n_epochs= 50
lr = 0.001
n_snr= 3

# In-domain case
for i in range(n_snr):
    source_mask = (Y[:, 1] == tar_snr)
    
    X_g = X[source_mask]
    Y_g = Y[source_mask]
    Y_g = Y_g[:,0]
    
    S_train_loader, S_val_loader = funcs.create_loader(X_g, Y_g, batch_size=128, permute=False)

    print('In-domain')
    _, t_golden = base.Base(
        model_cls=DeepResNet,
        device=device,
        S_train_loader=S_train_loader, 
        S_val_loader=S_val_loader,
        T_val_loader=S_val_loader,
        class_subset=class_subset, 
        n_classes=len(class_subset),
        lr=lr,
        n_epochs=n_epochs,
        n_runs=n_runs
    ).run()
    torch.cuda.empty_cache()
    #s_deep_coral_acc.append(s_deep)
    t_golden_acc.append(t_golden)

    tar_snr += 4
    

# UDA cases
tar_snr = 10
for i in range(n_snr):    
    source_mask = (Y[:, 1] == sou_snr)
    target_mask = (Y[:, 1] == tar_snr)
    
    X_s = X[source_mask]
    Y_s = Y[source_mask]
    Y_s = Y_s[:,0]
    
    X_t = X[target_mask]
    Y_t = Y[target_mask]
    Y_t = Y_t[:,0]

    
    # Dataloaders
    S_train_loader, S_val_loader = funcs.create_loader(X_s, Y_s, batch_size=128, permute=False)
    T_train_loader, T_val_loader = funcs.create_loader(X_t, Y_t, batch_size=128, permute=False)

    print('Base')
    _, t_base = base.Base(
        model_cls=DeepResNet,
        device=device,
        S_train_loader=S_train_loader, 
        S_val_loader=S_val_loader,
        T_val_loader=T_val_loader,
        class_subset=class_subset, 
        n_classes=len(class_subset),
        lr=lr,
        n_epochs=n_epochs,
        n_runs=n_runs
    ).run()
    torch.cuda.empty_cache()
    #s_deep_coral_acc.append(s_deep)
    t_base_acc.append(t_base)

    print('DANN')
    _, t_dann = dann.DAN(
        dann.DANN,
        FA=DANN_F,
        LP=DANN_LP,
        DC=DANN_DC,
        device=device,
        S_train_loader=S_train_loader,
        S_val_loader=S_val_loader,
        T_train_loader=T_train_loader,
        T_val_loader=T_val_loader,
        class_subset=class_subset,
        n_classes=len(class_subset),
        lr=lr,
        n_epochs=n_epochs,
        n_runs=n_runs
    ).run()
    torch.cuda.empty_cache()
    #s_deep_coral_acc.append(s_deep)
    t_dann_acc.append(t_dann)

    print('Deep CORAL')
    _, t_deep = DeepCORAL(
        G=CORAL_G, 
        C=CORAL_C, 
        device=device, 
        S_train_loader=S_train_loader,
        S_val_loader=S_val_loader,
        T_train_loader=T_train_loader,
        T_val_loader=T_val_loader,
        class_subset=class_subset,
        n_classes=len(class_subset),
        lr=lr, 
        n_epochs=n_epochs, 
        n_runs=n_runs,
        patience=5,
        lambda_coral=0.5,
        deep_weights=(1.0, 1.0, 1.0)
    ).run()
    torch.cuda.empty_cache()
    #s_deep_coral_acc.append(s_deep)
    t_deep_coral_acc.append(t_deep)

    print('STAR')
    _, t_star =  star.Star(
        G=STAR_G,
        C=STAR_C,
        device=device,
        S_train_loader=S_train_loader,
        S_val_loader=S_val_loader,  
        T_train_loader=T_train_loader,
        T_val_loader=T_val_loader,
        class_subset=class_subset,
        n_classes=len(class_subset),
        lr=lr,
        n_epochs=n_epochs,
        n_runs=n_runs,
        patience=5
    ).run()
    torch.cuda.empty_cache()
    #s_deep_coral_acc.append(s_deep)
    t_star_acc.append(t_star)

    print('MCD')
    _, t_mcd = mcd.Mcd(
        G=MCD_G,
        C=MCD_C,
        device=device,
        S_train_loader=S_train_loader,
        S_val_loader=S_val_loader,  
        T_train_loader=T_train_loader,
        T_val_loader=T_val_loader,
        class_subset=class_subset,
        n_classes=len(class_subset),
        lr=lr,
        n_epochs=n_epochs,
        n_runs=n_runs,
        patience=5
    ).run()
    torch.cuda.empty_cache()
    #s_deep_coral_acc.append(s_deep)
    t_mcd_acc.append(t_mcd)

    print('JAN')
    _, t_jan = jan.Jan(
        C=C_JAN,
        G=JAN_G,
        num_classes=len(class_subset),
        device=device,
        S_train_loader=S_train_loader,
        T_train_loader=T_train_loader,
        S_val_loader=S_val_loader,
        T_val_loader=T_val_loader,
        n_epochs=n_epochs,
        lr=lr,
        lambda_jmmd=0.1,
        n_runs=n_runs
    ).run()
    torch.cuda.empty_cache()
    #s_deep_coral_acc.append(s_deep)
    t_jan_acc.append(t_jan)
    
    tar_snr += 4

In [ ]:
x = np.arange(1, n_snr+1)
print(len(t_deep_coral_acc))
print(len(x))

plt.figure(figsize=(10, 6))
plt.plot(x, t_base_acc, marker='o', linestyle='-', label='Base')
plt.plot(x, t_dann_acc, marker='^', linestyle='--', label='DANN')
plt.plot(x, t_star_acc, marker='<', linestyle='--', label='Star')
plt.plot(x, t_mcd_acc, marker='D', linestyle='--', label='MCD')
plt.plot(x, t_deep_coral_acc, marker='v', linestyle='--', label='DCORAL')
plt.plot(x, t_jan_acc, marker='x', linestyle='--', label='JAN')
plt.plot(x, t_golden_acc, marker='>', linestyle='-', label='In-domain')

plt.xlabel('SNR (dB)')
plt.ylabel('Acc (%)')
plt.legend()
plt.grid(True)
plt.title('Simulated Data SNR-to-SNR DA Comparison')
plt.show()